In [3]:
from stable_baselines3 import A2C, PPO
from stable_baselines3.common.env_checker import check_env
from utils.DataLoader import DataLoader
from envs.CustomBuySellEnv import CustomBuySellEnv

data = DataLoader()
env = CustomBuySellEnv(data)
check_env(env)

model = A2C("MlpPolicy", env, verbose=1)
model.learn(total_timesteps=10000)

[*********************100%***********************]  1 of 1 completed
/home/tommy/ai-playground/.venv/lib/python3.12/site-packages/stable_baselines3/common/env_checker.py:37: UserWarning: Discrete action space  with a non-zero start (start=-1) is not supported by Stable-Baselines3. You can use a wrapper or update your action space.
  warnings.warn(


Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
---------------------------------------
| rollout/              |             |
|    ep_len_mean        | 364         |
|    ep_rew_mean        | -375        |
| time/                 |             |
|    fps                | 64          |
|    iterations         | 100         |
|    time_elapsed       | 7           |
|    total_timesteps    | 500         |
| train/                |             |
|    entropy_loss       | -1.07       |
|    explained_variance | 0.066603124 |
|    learning_rate      | 0.0007      |
|    n_updates          | 99          |
|    policy_loss        | -27.5       |
|    value_loss         | 1.07e+03    |
---------------------------------------
---------------------------------------
| rollout/              |             |
|    ep_len_mean        | 364         |
|    ep_rew_mean        | -417        |
| time/                 |             |
|    fps                | 

In [ ]:
from matplotlib import pyplot as plt
import pandas as pd
import quantstats as qs

test_data = DataLoader('BTC-USD', '2024-08-01', '2024-10-01')

test_env = CustomBuySellEnv(test_data)
check_env(test_env)

portfolio_values = []

observation, info = test_env.reset()

while True:
    action, _states = model.predict(observation)

    observation, reward, terminated, truncated, info = test_env.step(action)
    done = terminated or truncated
    portfolio_values.append(info['portfolio_value'])

    if done:
        break

actions_data = test_env.close()
actions_data.index = pd.to_datetime(test_data.frame.index)
buy_signals = actions_data[actions_data['Action'] == 1]
sell_signals = actions_data[actions_data['Action'] == -1]

# Plot the closing prices
plt.figure(figsize=(12, 6))
plt.plot(actions_data.index, test_data.frame['Close'], label='Close Price', color='blue', linewidth=1.5)

# Plot buy signals (green dots)
plt.scatter(buy_signals.index, test_data.frame.loc[buy_signals.index, 'Close'], color='green', label='Buy Signal', marker='o', s=10)

# Plot sell signals (red dots)
plt.scatter(sell_signals.index, test_data.frame.loc[sell_signals.index, 'Close'], color='red', label='Sell Signal', marker='o', s=10)

# Adding labels and legend
plt.xlabel('Date')
plt.ylabel('Close Price')
plt.title('Stock Price with Buy and Sell Signals')
plt.legend()
plt.grid(True)
plt.show()

dates = pd.date_range(start=test_data.frame.index[0], periods=len(portfolio_values), freq="D")
portfolio_series = pd.Series(portfolio_values, index=dates)

returns = portfolio_series.pct_change().fillna(0)

qs.reports.basic(returns)

[*********************100%***********************]  1 of 1 completed


NameError: name 'StockTradingEnv' is not defined